In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession, SQLContext
import os
from IPython.display import display
from IPython.display import clear_output


from datetime import datetime, time
from pyspark.sql.functions import year, month, dayofmonth, hour, minute


import pandas as pd
import folium


import ipywidgets as widgets
from ipywidgets import HBox, VBox, Box
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display


import ipywidgets as widgets
from ipywidgets import interact, interact_manual, DatePicker, Dropdown, Button



%matplotlib inline

spark = SparkSession.builder.master('local').appName('prova').getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

df2 = sqlContext.read.parquet(os.path.join(os.getcwd(), "sink", "sink_one_minute" , "*"))
df2.show()


+--------------------+---------+---------+-------------+-------------+-------------------+-------------------+
|                name|unique_id|id_object|average_count|average_speed|       window_start|         window_end|
+--------------------+---------+---------+-------------+-------------+-------------------+-------------------+
|Strada Nazionale ...|       20|        6|         59.0|          0.0|2020-07-10 18:11:00|2020-07-10 18:12:00|
|Via Giovan Battis...|        3|        5|         73.0|          0.0|2020-07-10 18:11:00|2020-07-10 18:12:00|
|Viale Antonio Gra...|       18|        6|         59.0|          0.0|2020-07-10 18:11:00|2020-07-10 18:12:00|
|Viale Antonio Gra...|       18|        6|         38.0|          0.0|2020-07-11 09:11:00|2020-07-11 09:12:00|
+--------------------+---------+---------+-------------+-------------+-------------------+-------------------+



In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession, SQLContext
import os
from IPython.display import display
from IPython.display import clear_output


from datetime import datetime, time
from pyspark.sql.functions import year, month, dayofmonth, hour, minute


import pandas as pd
import folium


import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display


import ipywidgets as widgets
from ipywidgets import interact, interact_manual, DatePicker, Dropdown, Button



%matplotlib inline


df = pd.read_csv(os.path.join(os.getcwd(),"punti_con_unique_id_e_coordinate.csv"), header=None)
crs = {'init': 'epsg:4326'}
BBox = (df[7].min(), df[7].max(), df[8].min(), df[8].max())
map = folium.Map(location=[44.655482, 10.930833], zoom_start=15, tiles="CartoDb dark_matter")

spark = SparkSession.builder.master('local').appName('prova').getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)

# Variabili globali
hours = '00'
minutes = '00'

def vehicle(x):
    return {
        'car': 6,
        'person': 14,
        'bus': 5,
        'motorbike': 13,
        'bicycle': 1,
    }[x]


def granularity(x):
    return {
        '1 minute': 'sink_one_minute',
        '15 minutes': 'sink_fifteen_minute',
        '1 hour': 'sink_one_hour',
        '1 day': 'sink_one_day',
    }[x]


def freq(x):
    return {
        '1 minute': '1min',
        '15 minutes': '15min',
        '1 hour': '1h',
        '1 day': '1d',
    }[x]

# Definizione dei widget
date_picker = DatePicker(description = "Pick a Date:")
display(date_picker)

mezzo = Dropdown(
    options=['person', 'car', 'bus', 'motorbike', 'bicycle'],
    value='car',
    description='Object:',
    disabled=False,
)

display(mezzo)

granularità = Dropdown(
    options= ['1 minute', '15 minutes', '1 hour', '1 day'],
    value='1 minute',
    description='Granularity:',
    disabled=False,
)

# Inizializzo lo slider come un testo, poi verrà modificato successivamente nelle funzioni
selection_range_slider1 = widgets.Text()

# Funzione che definisce cosa fare nel momento in cui viene modificata la granularità
output_selected = widgets.Text() 
def on_change(change):
    global hours
    global minutes
    global selection_range_slider1

    start_time = time(0,0)
    end_time = time(23, 59)
    
    # Cambia scala dello slider e imposta il valore 00:00
    new_times = pd.date_range(str(start_time), str(end_time), freq=freq(granularità.value)).strftime('%H:%M')
    new_options = [date for date in new_times]
    selection_range_slider1.options = new_options
    hours = '00'
    minutes = '00'
    
granularità.observe(on_change, names=['value'])

display(granularità)

btn = Button(description = "Go") 
display(btn)

# Funzione che modifica e colora i punti sulla mappa
def customFunction(row):
    # interrogo dataframe location
    if row.name != "null":
        a = df.loc[(df[2] == row.name) & (df[6] == int(row.unique_id))]

        # Colora i punti sul grafico
        traffic_color = "#00FF00"
        if row.average_count != None:
            if 6 <= row.average_count < 13:
                traffic_color = "#FF8040"
            if row.average_count >= 13:
                traffic_color = "#FF0000"
        folium.CircleMarker(location=(a.iloc[0][8], a.iloc[0][7]), color = traffic_color, radius=2).add_to(map)


    
    
# Fuznione che viene attivata quando il pulsante 'Go' è premuto
def show_map_and_n_point(self):
    if date_picker.value is not None:
        global hours
        global minutes
        global selection_range_slider1


        # Pulisco output precedente
        clear_output()
        display(date_picker)
        display(mezzo)
        display(granularità)
        display(btn)

        # Ottieni il tipo di veicolo
        type_vehicle = vehicle(mezzo.value)

        # Ottieni la granularità
        type_granularity = granularity(granularità.value)

        # Ottieni la data
        type_date = date_picker.value

        # Ottieni la frequenza per lo slider
        type_freq = freq(granularità.value)

        # Slider tempo
        start_time = time(0,0)
        end_time = time(23, 59)

        times = pd.date_range(str(start_time), str(end_time), freq=freq(granularità.value)).strftime('%H:%M')
        options = [date for date in times]

        selection_range_slider1 = widgets.SelectionSlider(
            options=options,
            value = hours+':'+minutes,
            description='Times',
            orientation='horizontal',
            continuous_update=False,
            layout={'width': '500px'}
        )


        display(selection_range_slider1)

        # Funzione che viene attivata nel momento in cui è modificato il valore nello slider
        def on_value_change(change):
            global hours
            global minutes
            # Ottieni le ore e i minuti impostati
            output_selected.value = selection_range_slider1.value    
            hours, minutes = output_selected.value.split(':')
            show_map_and_n_point(self)


        selection_range_slider1.observe(on_value_change, names='value')


        if type_date is not None:     
            locs = zip(df[8], df[7])
            for location in locs:
                traffic_color = "#00FF00"
                folium.CircleMarker(location=(location[0], location[1]), color = traffic_color, radius=2).add_to(map)

            # Se ho selezionato una data corretta
            # Carica il corretto file parquet
            df2 = sqlContext.read.parquet(os.path.join(os.getcwd(), "sink", type_granularity , "*"))
            
            # Se la granularità è un giorno non contare minuti e ore 
            if granularità.value == '1 day':
             base_df = df2.select('average_count', 'name', 'unique_id', 'window_start')\
                .where(df2.id_object == type_vehicle)\
                .where(dayofmonth(df2.window_start) == type_date.day)
            else:
                # Ritorna tutte le righe che hanno veicolo, data e granularità corrette
                base_df = df2.select('average_count', 'name', 'unique_id', 'window_start')\
                    .where(df2.id_object == type_vehicle)\
                    .where(dayofmonth(df2.window_start) == type_date.day)\
                    .where(month(df2.window_start) == type_date.month)\
                    .where(hour(df2.window_start) == hours)\
                    .where(minute(df2.window_start) == minutes)

            # Per ogni punto ritornato dalla query, coloralo del colore giusto
            for x in base_df.rdd.collect():
                customFunction(x)

            # Mostra mappa
            display(map)
    else:
        display(widgets.HBox([widgets.Label(value="Pick a date!")]))

btn.on_click(show_map_and_n_point)

DatePicker(value=datetime.date(2020, 7, 24), description='Pick a Date:')

Dropdown(description='Object:', index=1, options=('person', 'car', 'bus', 'motorbike', 'bicycle'), value='car'…

Dropdown(description='Granularity:', options=('1 minute', '15 minutes', '1 hour', '1 day'), value='1 minute')

Button(description='Go', style=ButtonStyle())

SelectionSlider(continuous_update=False, description='Times', layout=Layout(width='500px'), options=('00:00', …